In [1]:
import torch.nn as nn
import torch.optim as optim
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import shap
from joblib import dump, load
import warnings

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
loaded_datasets_info = torch.load('/Users/jiaming/Desktop/Lab2/datas/saved_datasets_scaled.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

In [3]:
from torch.utils.data import DataLoader

def extract_features_labels_from_subset(subset):
    
    loader = DataLoader(subset, batch_size=len(subset))
    
    for features, labels in loader:
        features = features.squeeze(1).numpy()
        labels = labels.squeeze(1).numpy()
        return features, labels

X_train, y_train = extract_features_labels_from_subset(loaded_train_dataset)
X_val, y_val = extract_features_labels_from_subset(loaded_val_dataset)
X_test, y_test = extract_features_labels_from_subset(loaded_test_dataset)

In [8]:
pos = pd.read_csv('/Users/jiaming/Desktop/Lab2/datas/neg.csv') # 644
feature_names = pos.columns

# 4ML (LR + NB + RF + XGB)

In [86]:
# LR -> LinearExplainer (need model, masker)
# NB -> KernelExplainer (need model, shap.kmeans)
# RF -> TreeExplainer (need model)
# XGB -> Explainer (need model)

# for constructing explainer, use train set
# for get shap values with explainer, use test set

## 1. LR

In [112]:
lr_model = load('/Users/jiaming/Desktop/Lab2/datas/ROC/models/lr_model.joblib')

masker = shap.maskers.Independent(data=X_train)
explainer_lr = shap.LinearExplainer(lr_model, masker)  
shap_values_lr = explainer_lr(X_test)

shap_values_lr_reconstructed = shap.Explanation(values=shap_values_lr, 
                                             base_values=explainer_lr.expected_value, 
                                             data=X_test, 
                                             feature_names=feature_names)

fig1_lr = shap.plots.bar(shap_values_lr_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig1_lr.pdf', bbox_inches='tight')
plt.close(fig1_lr) 

fig2_lr = shap.plots.beeswarm(shap_values_lr_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig2_lr.pdf', bbox_inches='tight')
plt.close(fig2_lr) 

## 2. NB

In [88]:
# nb_model = load('/Users/jiaming/Desktop/Lab2/datas/ROC/models/nb_model.joblib') # has some issues

nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

explainer_nb = shap.KernelExplainer(nb_model.predict_proba, shap.kmeans(X_train, 10))  
shap_values_nb = explainer_nb(X_test)

shap_values_nb_reconstructed = shap.Explanation(values=shap_values_nb.values[:,:,0], 
                                             base_values=explainer_nb.expected_value[0], 
                                             data=X_test, 
                                             feature_names=feature_names)

fig1_nb = shap.plots.bar(shap_values_nb_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig1_nb.pdf', bbox_inches='tight')
plt.close(fig1_nb) 

fig2_nb = shap.plots.beeswarm(shap_values_nb_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig2_nb.pdf', bbox_inches='tight')
plt.close(fig2_nb) 

  0%|          | 0/258 [00:00<?, ?it/s]

## 3. RF

In [89]:
rf_model = load('/Users/jiaming/Desktop/Lab2/datas/ROC/models/rf_model.joblib')

explainer_rf = shap.TreeExplainer(rf_model)
shap_values_rf = explainer_rf(X_test)

shap_values_rf_reconstructed = shap.Explanation(values=shap_values_rf.values[:,:,0], 
                                             base_values=explainer_rf.expected_value[0], 
                                             data=X_test, 
                                             feature_names=feature_names)

fig1_rf = shap.plots.bar(shap_values_rf_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig1_rf.pdf', bbox_inches='tight')
plt.close(fig1_rf) 

fig2_rf = shap.plots.beeswarm(shap_values_rf_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig2_rf.pdf', bbox_inches='tight')
plt.close(fig2_rf) 

## 4. SVM

In [13]:
svm_model = load('/Users/jiaming/Desktop/Lab2/datas/ROC/models/svm_model.joblib')

explainer_svm = shap.KernelExplainer(svm_model.predict_proba, shap.kmeans(X_train, 10))  
shap_values_svm = explainer_svm(X_test)

shap_values_svm_reconstructed = shap.Explanation(values=shap_values_svm.values[:,:,0], 
                                             base_values=explainer_svm.expected_value[0], 
                                             data=X_test, 
                                             feature_names=feature_names)

fig1_svm = shap.plots.bar(shap_values_svm_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig1_svm.pdf', bbox_inches='tight')
plt.close(fig1_svm) 

fig2_svm = shap.plots.beeswarm(shap_values_svm_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig2_svm.pdf', bbox_inches='tight')
plt.close(fig2_svm) 

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


  0%|          | 0/258 [00:00<?, ?it/s]

No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


# 2DL (LSTM + ResNet)

In [91]:
loaded_datasets_info = torch.load('/Users/jiaming/Desktop/Lab2/datas/saved_datasets_scaled.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

In [92]:
from torch.utils.data import DataLoader

def extract_features_labels_from_subset(subset):
    
    loader = DataLoader(subset, batch_size=len(subset))
    
    for features, labels in loader:
        features = features.squeeze(1).numpy()
        labels = labels.squeeze(1).numpy()
        return features, labels

X_train, y_train = extract_features_labels_from_subset(loaded_train_dataset)
X_val, y_val = extract_features_labels_from_subset(loaded_val_dataset)
X_test, y_test = extract_features_labels_from_subset(loaded_test_dataset)

In [93]:
# for deep learning model, here we expand 2nd dimension (channel) to 1 
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1) 
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

## 1. LSTM

In [94]:
class BinaryLSTM(nn.Module):
    def __init__(self, input_size=24, hidden_size=256, num_layers=2):
        super(BinaryLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.linear(lstm_out[:, -1, :])
        output = self.sigmoid(output)
        return output

lstm_model = BinaryLSTM()
lstm_model.load_state_dict(torch.load('/Users/jiaming/Desktop/Lab2/datas/ROC/models/LSTM/LSTM.pth'))

<All keys matched successfully>

In [95]:
# warnings.filterwarnings("ignore") # this code chunk will have warnings every time constructing the DeepExplainer

explainer_lstm = shap.DeepExplainer(lstm_model, X_train_tensor) # X_train_tensor as background_data
shap_values_lstm = explainer_lstm.shap_values(X_test_tensor) # 1. shap values

lstm_model.eval()
with torch.no_grad():
    predictions = lstm_model(background_data)
expected_value = predictions.numpy() # 2. base_values (which is just the expected value)

shap_values_lstm_reconstructed = shap.Explanation(values=shap_values_lstm.squeeze(1), # construct back to only 2 dim
                                             base_values=expected_value.squeeze(1), 
                                             data=X_test_tensor.squeeze(1), # construct back to only 2 dim
                                             feature_names=feature_names)

In [96]:
fig1_lstm = shap.plots.bar(shap_values_lstm_reconstructed, show=False)
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig1_lstm.pdf', bbox_inches='tight')
plt.close(fig1_lstm)

fig2_lstm = shap.plots.beeswarm(shap_values_lstm_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig2_lstm.pdf', bbox_inches='tight')
plt.close(fig2_lstm) 

## 2. ResNet

In [97]:
size = 8
class Net_conv(torch.nn.Module):
    def __init__(self, input_length):
        super(Net_conv, self).__init__()
        self.block_1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=1,
                            out_channels=size,
                            kernel_size=1,
                            stride=1,
                            padding=0),
            torch.nn.BatchNorm1d(size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(in_channels=size,
                                out_channels=2*size,
                                kernel_size=3,
                                stride=1,
                                padding=1),
            torch.nn.BatchNorm1d(2*size)
        )
        self.block_2 = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=2*size,
                            out_channels=4*size,
                            kernel_size=1,
                            stride=1,
                            padding=0),
            torch.nn.BatchNorm1d(4*size),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(in_channels=4*size,
                                out_channels=2*size,
                                kernel_size=3, 
                                stride=1,
                                padding=1),
            torch.nn.BatchNorm1d(2*size)
        )
        iutput_size_block_1 = (input_length - 1 + 2 * 0) // 1 + 1  
        output_size_block_2 = (iutput_size_block_1 - 1 + 2 * 0) // 1 + 1  
        num_channels_last_layer = 2*size 
        linear_input_size = num_channels_last_layer * output_size_block_2    
        self.linear_1 = torch.nn.Linear(linear_input_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        shortcut = x.float()
        x = self.block_1(x)
        x = torch.nn.functional.relu(x + shortcut)    
        shortcut = x
        x = self.block_2(x)
        x = torch.nn.functional.relu(x + shortcut)     
        x = x.view(x.size(0), -1)
        x =  self.linear_1(x)
        x = self.sigmoid(x)
        return x

resnet_model = Net_conv(input_length = 24)
resnet_model.load_state_dict(torch.load('/Users/jiaming/Desktop/Lab2/datas/ROC/models/ResNet/ResNet.pth'))

<All keys matched successfully>

In [98]:
# warnings.filterwarnings("ignore") # this code chunk will have warnings every time constructing the DeepExplainer

explainer_resnet = shap.DeepExplainer(resnet_model, X_train_tensor) # X_train_tensor as background_data
shap_values_resnet = explainer_resnet.shap_values(X_test_tensor) # 1. shap values

resnet_model.eval()
with torch.no_grad():
    predictions = resnet_model(background_data)
expected_value = predictions.numpy() # 2. base_values (which is just the expected value)

shap_values_resnet_reconstructed = shap.Explanation(values=shap_values_resnet.squeeze(1), # construct back to only 2 dim
                                             base_values=expected_value.squeeze(1), 
                                             data=X_test_tensor.squeeze(1), # construct back to only 2 dim
                                             feature_names=feature_names)

In [100]:
fig1_resnet = shap.plots.bar(shap_values_resnet_reconstructed, show=False)
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig1_resnet.pdf', bbox_inches='tight')
plt.close(fig1_resnet)

fig2_resnet = shap.plots.beeswarm(shap_values_resnet_reconstructed, show=False) 
plt.savefig('/Users/jiaming/Desktop/Lab2/datas/SHAP/shap_fig2_resnet.pdf', bbox_inches='tight')
plt.close(fig2_resnet) 